In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json

In [2]:
#set the keys
yelp_id = os.environ["YELP_FUSION_API"]
print(yelp_id)

i35ggGDg9T4paIS3DKh6l3zcKaOd4DLdtThZb_rB4OZks2jet4BAlAaKSHLCO5mT3LYrajUpxspPwQBX9_3ssO29_vnHKCh57vCbhpQJlCiUdkjsiCpXUoX18x7EZHYx


In [3]:
class yelp_api_request:
    """
    Get a list of parameters to construct api request
    """
    def __init__(self, api_key, search_path, term, location, search_limit, latitude, longitude):
        self.api_key = api_key
        self.api_host = 'https://api.yelp.com'
        self.search_path = search_path
        self.term = term
        self.location = location
        self.search_limit = search_limit
        self.latitude = latitude
        self.longitude = longitude

    def request(self, url_params=None):
        """Given your API_KEY, send a GET request to the API.
        Args:
            host (str): The domain host of the API.
            path (str): The path of the API after the domain.
            API_KEY (str): Your API Key.
            url_params (dict): An optional set of query parameters in the request.
        Returns:
            dict: The JSON response from the request.
        """
        url = '{0}{1}'.format(self.api_host, self.search_path)
        headers = {
            'Authorization': "Bearer %s" % self.api_key,
        }
        response = requests.request('GET', url, headers=headers, params=url_params)
        return response.json()

    def search(self):
        """Query the Search API by a search term and location.
        Args:
            term (str): The search term passed to the API.
            location (str): The search location passed to the API.

        Returns:
            dict: The JSON response from the request.
        """
        url_params = {
            'term': self.term.replace(' ', '+'),
            # 'location': self.location.replace(' ', '+'),
            'limit': self.search_limit,
            'latitude': self.latitude,
            'longitude': self.longitude,
            'radius': 1000,
            'text': 'bike'
        }
        return self.request(url_params=url_params)

In [57]:
API_KEY = os.environ["YELP_FUSION_API"]

SEARCH_PATH = '/v3/businesses/search'
# SEARCH_PATH = '/v3/autocomplete'

# Defaults for our simple example.
DEFAULT_TERM = 'park'
DEFAULT_LOCATION = 'Toronto, CA'
LATITUDE = "43.665269"
LONGITUDE = "-79.319796"
SEARCH_LIMIT = 20

response = yelp_api_request(API_KEY, SEARCH_PATH, DEFAULT_TERM, DEFAULT_LOCATION, SEARCH_LIMIT, LATITUDE, LONGITUDE).search()
response



{'businesses': [{'id': 'Drn62oroI35wMeg1aMWlCw',
   'alias': 'woodbine-park-toronto',
   'name': 'Woodbine Park',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/RPuAR4UOseTVor-suCi9bw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/woodbine-park-toronto?adjust_creative=C3SB6jN-CFWUr0P-X-hVGg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=C3SB6jN-CFWUr0P-X-hVGg',
   'review_count': 10,
   'categories': [{'alias': 'parks', 'title': 'Parks'}],
   'rating': 4.0,
   'coordinates': {'latitude': 43.6633877134187,
    'longitude': -79.3128788358741},
   'transactions': [],
   'location': {'address1': 'Coxwell Ave & Lake Shore Blvd E',
    'address2': '',
    'address3': '',
    'city': 'Toronto',
    'zip_code': 'M4L',
    'country': 'CA',
    'state': 'ON',
    'display_address': ['Coxwell Ave & Lake Shore Blvd E',
     'Toronto, ON M4L',
     'Canada']},
   'phone': '',
   'display_phone': '',
   'distance': 594.4277162889941},
  {'id': 'BH

In [81]:
for i in response:
    print(i)
response['businesses'][0]['categories'][0]
category_list = []
for i in response['businesses']:
    category_list.append(i['categories'][-1:][0]['title'])
category_list
park_db = pd.json_normalize(response['businesses'])
park_db.drop(columns=['categories'])
park_db['categories'] = category_list
park_db.columns

businesses
total
region


Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'phone', 'display_phone',
       'distance', 'coordinates.latitude', 'coordinates.longitude',
       'location.address1', 'location.address2', 'location.address3',
       'location.city', 'location.zip_code', 'location.country',
       'location.state', 'location.display_address'],
      dtype='object')

In [10]:
# Get all the information from the file, and create a new column, "ll", that records the latitude and longitude.
bike_dataframe = pd.read_csv('bikes.csv')
bike_dataframe = bike_dataframe[['empty_slots', 'free_bikes', 'name', 'latitude', 'longitude']]
bike_dataframe['ll'] = bike_dataframe[['latitude', 'longitude']].apply(lambda x: f"{x['latitude']}, {x['longitude']}", axis=1)
bike_dataframe


,empty_slots,free_bikes,name,latitude,longitude,ll
0,3,16,Queen St E / Woodward Ave,43.665269,-79.319796,"43.665269, -79.319796"
1,13,2,Primrose Ave / Davenport Rd,43.671420,-79.445947,"43.67142, -79.445947"
2,9,14,Queen St. E / Rhodes Ave.,43.666224,-79.317693,"43.666224, -79.317693"
3,9,15,Bond St / Queen St E,43.653236,-79.376716,"43.653236, -79.376716"
4,9,18,Church St / Alexander St,43.663722,-79.380288,"43.663722, -79.380288"
...,...,...,...,...,...,...
669,11,3,Parkcrest Dr / Kingston Rd,43.735967,-79.219952,"43.73596719224253, -79.21995221481933"
670,18,1,Scarborough Soccer Centre,43.715466,-79.287660,"43.7154661, -79.2876595"
671,13,1,Lansdowne Ave / Seaforth Ave,43.642904,-79.437461,"43.64290439595669, -79.43746059447746"
672,15,0,Danforth Rd / Birchmount Rd,43.706880,-79.268510,"43.70688, -79.26850999999999"
